In [1]:
import pandas as pd
import numpy as np
import json
import datetime
from pandas.io.json import json_normalize
import scipy as sp
import scipy.fftpack
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io import wavfile
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import os

In [2]:
sr_data, data = wavfile.read('S04_U01.CH1_demo.wav')
x , sr = librosa.load('S04_U01.CH1_demo.wav', sr = sr_data)
print(x)
print(len(x))
print(sr)

[0.02331543 0.03274536 0.02090454 ... 0.00289917 0.02050781 0.02075195]
9600000
16000


In [7]:
def json_to_df():
    #load json file
    with open('S04.json','r') as f:
        data = json.loads(f.read()) 
    df = pd.json_normalize(data)

    #format timestamps as datetime objects
    df['start_time.U01'] = pd.to_datetime(df['start_time.U01']) 
    df['end_time.U01'] = pd.to_datetime(df['end_time.U01']) 

    #filter to specific 10 minute part
    part_start_time_U01 = pd.to_datetime('01:29:59')
    part_end_time_U01 = pd.to_datetime('01:39:57')
    df_new = df.loc[(df['start_time.U01'] > part_start_time_U01) & (df['start_time.U01'] < part_end_time_U01)]

    # final dataframe
    df_new = df_new.filter(['words','speaker', 'session_id','start_time.U01', 'end_time.U01'], axis=1)
    return df_new
    


In [8]:
df = json_to_df()
print(df.head())

                                                  words speaker session_id  \
3177  It's basically to the end of your passport alm...     P10        S04   
3178                      Short of like, a half a year.     P10        S04   
3179             Maybe like five or six years from now?     P11        S04   
3180                          Then yeah, it's worth it.     P10        S04   
3181  Like, if your passports expire the next two ye...     P10        S04   

              start_time.U01            end_time.U01  
3177 2021-09-29 01:29:59.910 2021-09-29 01:30:01.840  
3178 2021-09-29 01:30:02.240 2021-09-29 01:30:03.740  
3179 2021-09-29 01:30:04.680 2021-09-29 01:30:06.000  
3180 2021-09-29 01:30:05.940 2021-09-29 01:30:07.060  
3181 2021-09-29 01:30:07.060 2021-09-29 01:30:09.610  


In [13]:
part_start_time_U01 = pd.to_datetime('01:29:59')
half_sec = sr/2
print(half_sec)
test = x[:int(half_sec)]
temp = part_start_time_U01 + datetime.timedelta(seconds=0.5)
temp_2 = pd.to_datetime('01:29:59.910')
print(test)
print(len(test))
print(temp)
print(df['start_time.U01'])
type(df)
type(df['start_time.U01'])


8000.0
[ 0.02331543  0.03274536  0.02090454 ...  0.05657959 -0.07241821
 -0.04711914]
8000
2021-09-29 01:29:59.500000
3177   2021-09-29 01:29:59.910
3178   2021-09-29 01:30:02.240
3179   2021-09-29 01:30:04.680
3180   2021-09-29 01:30:05.940
3181   2021-09-29 01:30:07.060
                 ...          
3586   2021-09-29 01:39:52.470
3587   2021-09-29 01:39:54.100
3588   2021-09-29 01:39:54.260
3589   2021-09-29 01:39:55.760
3590   2021-09-29 01:39:56.830
Name: start_time.U01, Length: 414, dtype: datetime64[ns]


pandas.core.series.Series